In [1]:

# Import Libraries
import datetime as dt
import json
import os

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import src.data_processing.MIMIC.test_functions as tests

# LOAD CONFIGURATION 
with open("src/data_processing/MIMIC/MIMIC_PROCESSING_DEFAULT_VARS.json", "r") as f:
    DEFAULT_CONFIG = json.load(f)
    f.close()

if not os.path.exists(DEFAULT_CONFIG["SAVE_FD"]):
    os.makedirs(DEFAULT_CONFIG["SAVE_FD"])

In [9]:
try:
    assert os.path.exists(DEFAULT_CONFIG["SAVE_FD"] + "admissions_intermediate.csv")
    assert os.path.exists(DEFAULT_CONFIG["SAVE_FD"] + "vitals_intermediate.csv")

except AssertionError as e:
    raise e

In [13]:
adm_proc = pd.read_csv(
        DEFAULT_CONFIG["SAVE_FD"] + "admissions_intermediate.csv",
        index_col=0, 
        header=0, 
        parse_dates=["intime", "outtime", "intime_next", "outtime_next", "deathtime"]
    )
vit_proc = (
    pd.read_csv(
    DEFAULT_CONFIG["SAVE_FD"] + "vitals_intermediate.csv", 
    index_col=0, 
    header=0, 
    parse_dates=DEFAULT_CONFIG["VITALS_TIME_VARS"]
    )
    .reset_index(drop=False)
    .assign(sampled_time_to_end=lambda x: pd.to_timedelta(x["sampled_time_to_end"]))  # pd does not load timedelta automatically
)


# Load core info
transfers_core = pd.read_csv(
    DEFAULT_CONFIG["DATA_FD"] + "core/transfers.csv", 
    index_col=None, 
    header=0, 
    parse_dates=["intime", "outtime"]
)
admissions_core = pd.read_csv(
    DEFAULT_CONFIG["DATA_FD"] + "core/admissions.csv",
    index_col=None,
    header=0,
    parse_dates=["admittime", "dischtime", "deathtime", "edregtime", "edouttime"]
)

In [16]:
vit_proc.isna().sum(axis=0)

sampled_time_to_end        0
TEMP                   26487
HR                     21305
RR                     21420
SPO2                   21749
SBP                    21485
DBP                    21485
subject_id                 0
stay_id                    0
charttime                  0
rhythm                 51998
pain                    4341
hadm_id                  107
transfer_id_next       22632
eventtype_next         22632
careunit_next          22648
intime_next            22632
outtime_next           22632
transfer_id                0
eventtype                  0
careunit                   0
intime                     0
outtime                    0
gender                     0
age                        0
ESI                        0
deathtime              52433
dtype: int64